# Identifying safe loans with decision trees

The [LendingClub](https://www.lendingclub.com/) is a peer-to-peer leading company that directly connects borrowers and potential lenders/investors. In this notebook, you will build a classification model to predict whether or not a loan provided by LendingClub is likely to [default](https://en.wikipedia.org/wiki/Default_(finance)).

In this notebook you will use data from the LendingClub to predict whether a loan will be paid off in full or the loan will be [charged off](https://en.wikipedia.org/wiki/Charge-off) and possibly go into default. In this assignment you will:

* Use SFrames to do some feature engineering.
* Train a decision-tree on the LendingClub dataset.
* Visualize the tree.
* Predict whether a loan will default along with prediction probabilities (on a validation set).
* Train a complex tree model and compare it to simple tree model.

Let's get started!

## Fire up Graphlab Create

Make sure you have the latest version of GraphLab Create. If you don't find the decision tree module, then you would need to upgrade GraphLab Create using

```
   pip install graphlab-create --upgrade
```

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load LendingClub dataset

We will be using a dataset from the [LendingClub](https://www.lendingclub.com/). A parsed and cleaned form of the dataset is availiable [here](https://github.com/learnml/machine-learning-specialization-private). Make sure you **download the dataset** before running the following command.

In [2]:
loans = pd.read_csv('lending-club-data.csv')

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Exploring some features

Let's quickly explore what the dataset looks like. First, let's print out the column names to see what features we have in this dataset.

In [3]:
loans.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc',
       'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'not_compliant', 'status', 'inactive_loans',
       'bad_loans', 'emp_length_num', 'grade_num', '

Here, we see that we have some feature columns that have to do with grade of the loan, annual income, home ownership status, etc. Let's take a look at the distribution of loan grades in the dataset.

In [4]:
loans['grade'][:10]

0    B
1    C
2    C
3    C
4    A
5    E
6    F
7    B
8    C
9    B
Name: grade, dtype: object

We can see that over half of the loan grades are assigned values `B` or `C`. Each loan is assigned one of these grades, along with a more finely discretized feature called `sub_grade` (feel free to explore that feature column as well!). These values depend on the loan application and credit report, and determine the interest rate of the loan. More information can be found [here](https://www.lendingclub.com/public/rates-and-fees.action).

Now, let's look at a different feature.

In [5]:
loans['home_ownership']

0             RENT
1             RENT
2             RENT
3             RENT
4             RENT
5             RENT
6              OWN
7             RENT
8              OWN
9              OWN
10            RENT
11            RENT
12            RENT
13            RENT
14            RENT
15        MORTGAGE
16        MORTGAGE
17            RENT
18            RENT
19             OWN
20            RENT
21            RENT
22        MORTGAGE
23            RENT
24            RENT
25        MORTGAGE
26            RENT
27        MORTGAGE
28        MORTGAGE
29            RENT
            ...   
122577        RENT
122578    MORTGAGE
122579    MORTGAGE
122580        RENT
122581        RENT
122582    MORTGAGE
122583        RENT
122584        RENT
122585         OWN
122586    MORTGAGE
122587    MORTGAGE
122588         OWN
122589    MORTGAGE
122590         OWN
122591    MORTGAGE
122592        RENT
122593    MORTGAGE
122594         OWN
122595    MORTGAGE
122596    MORTGAGE
122597    MORTGAGE
122598    MO

This feature describes whether the loanee is mortaging, renting, or owns a home. We can see that a small percentage of the loanees own a home.

## Exploring the target column

The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column **1** means a risky (bad) loan **0** means a safe  loan.

In order to make this more intuitive and consistent with the lectures, we reassign the target to be:
* **+1** as a safe  loan, 
* **-1** as a risky (bad) loan. 

We put this in a new column called `safe_loans`.

In [6]:
# safe_loans =  1 => safe
# safe_loans = -1 => risky
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.drop('bad_loans',1)

Now, let us explore the distribution of the column `safe_loans`. This gives us a sense of how many safe and risky loans are present in the dataset.

In [7]:
loans['safe_loans'].value_counts()/loans.shape[0]
#loans.shape

 1    0.811185
-1    0.188815
Name: safe_loans, dtype: float64

You should have:
* Around 81% safe loans
* Around 19% risky loans

It looks like most of these loans are safe loans (thankfully). But this does make our problem of identifying risky loans challenging.

## Features for the classification algorithm

In this assignment, we will be using a subset of features (categorical and numeric). The features we will be using are **described in the code comments** below. If you are a finance geek, the [LendingClub](https://www.lendingclub.com/) website has a lot more details about these features.

In [8]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                   # prediction target (y) (+1 means safe, -1 is risky)

# Extract the feature columns and target column
loans = loans[features + [target]]

What remains now is a **subset of features** and the **target** that we will use for the rest of this notebook. 

## Sample data to balance classes

As we explored above, our data is disproportionally full of safe loans.  Let's create two datasets: one with just the safe loans (`safe_loans_raw`) and one with just the risky loans (`risky_loans_raw`).

In [9]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print("Number of safe loans  : %s" % len(safe_loans_raw))
print("Number of risky loans : %s" % len(risky_loans_raw))

Number of safe loans  : 99457
Number of risky loans : 23150


Now, write some code to compute below the percentage of safe and risky loans in the dataset and validate these numbers against what was given using `.show` earlier in the assignment:

In [10]:
print("Percentage of safe loans  : %s" % (len(safe_loans_raw)/len(loans))) 
print("Percentage of risky loans : %s" % (len(risky_loans_raw)/len(loans)))


Percentage of safe loans  : 0.8111853319957262
Percentage of risky loans : 0.18881466800427382


One way to combat class imbalance is to undersample the larger class until the class distribution is approximately half and half. Here, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed=1` so everyone gets the same results.

In [11]:
# # Since there are fewer risky loans than safe loans, find the ratio of the sizes
# # and use that percentage to undersample the safe loans.
# percentage = len(risky_loans_raw)/float(len(safe_loans_raw))

# risky_loans = risky_loans_raw
# safe_loans = safe_loans_raw.sample(percentage, seed=1)

# Append the risky_loans with the downsampled version of safe_loans
# loans_data = risky_loans.append(safe_loans)

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [12]:
#print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
#print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
#print "Total number of loans in our new dataset :", len(loans_data)

**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## One-hot encoding

In [13]:
categorical_variables = []
loans_str = loans.select_dtypes(include = ['object'])
for feat_name in loans_str.columns.values.tolist():
    categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_unpacked = pd.get_dummies(loans[feature],prefix=feature)
    loans.drop(feature,axis=1,inplace=True)
    loans[loans_data_unpacked.columns.values.tolist()]=loans_data_unpacked   
    

## Split data into training and validation sets

In [14]:
# Load JSON files (see Assignement 2 of Week2)
import json
with open('module-5-assignment-1-train-idx.json', 'r') as f: 
    train_idx = json.load(f)
with open('module-5-assignment-1-validation-idx.json', 'r') as f: 
    validation_idx = json.load(f)

#train_data = products.iloc[train_idx,:]
#validation_data = products.iloc[validation_idx]

train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]

Now, let's verify that the resulting percentage of safe and risky loans are each nearly 50%.

In [15]:
train_data.shape
train_data.head()
print("Percentage of safe loans                 :", len(train_data[train_data['safe_loans']==1]) / float(len(train_data)))
print("Percentage of risky loans                :", len(train_data[train_data['safe_loans']==-1]) / float(len(train_data)))
print("Total number of loans in our new dataset :", (len(train_data)+len(validation_data)))

Percentage of safe loans                 : 0.5036535568450462
Percentage of risky loans                : 0.4963464431549538
Total number of loans in our new dataset : 46508


# Use decision tree to build a classifier

Now, let's use the built-in GraphLab Create decision tree learner to create a loan prediction model on the training data. (In the next assignment, you will implement your own decision tree learning algorithm.)  Our feature columns and target column have already been decided above. Use `validation_set=None` to get the same results as everyone else.

In [20]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
train_data = train_data.sort_values(by = target, ascending = True)
y = train_data[target].as_matrix()
print(y)
X = train_data.ix[:,train_data.columns!=target].as_matrix()
decision_tree_model=DecisionTreeClassifier(max_depth=6)
decision_tree_model.fit(X,y)


[-1 -1 -1 ...,  1  1  1]


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

As noted in the [documentation](https://dato.com/products/create/docs/generated/graphlab.boosted_trees_classifier.create.html#graphlab.boosted_trees_classifier.create), typically the max depth of the tree is capped at 6. However, such a tree can be hard to visualize graphically.  Here, we instead learn a smaller model with **max depth of 2** to gain some intuition by visualizing the learned tree.

In [21]:
small_model=DecisionTreeClassifier(max_depth=2)
small_model.fit(X,y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

# Making predictions

Let's consider two positive and two negative examples **from the validation set** and see what the model predicts. We will do the following:
* Predict whether or not a loan is safe.
* Predict the probability that a loan is safe.

In [22]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
19,0,11,11.18,1,1,82.4,0.0,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
79,0,10,16.85,1,1,96.4,0.0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
24,0,3,13.97,0,1,59.5,0.0,-1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
41,0,11,16.33,1,1,62.1,0.0,-1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Explore label predictions

Now, we will use our model  to predict whether or not a loan is likely to default. For each row in the **sample_validation_data**, use the **decision_tree_model** to predict whether or not the loan is classified as a **safe loan**. 

**Hint:** Be sure to use the `.predict()` method.

In [23]:
newX=sample_validation_data.ix[:,sample_validation_data.columns!=target].as_matrix()
sample_validation_data['pred'] = decision_tree_model.predict(newX)
print(sample_validation_data[['pred','safe_loans']])

    pred  safe_loans
19     1           1
79    -1           1
24    -1          -1
41     1          -1


**Quiz Question:** What percentage of the predictions on `sample_validation_data` did `decision_tree_model` get correct?

## Explore probability predictions

For each row in the **sample_validation_data**, what is the probability (according **decision_tree_model**) of a loan being classified as **safe**? 


**Hint:** Set `output_type='probability'` to make **probability** predictions using **decision_tree_model** on `sample_validation_data`:

In [32]:
sample_validation_data['pred_proba'] = decision_tree_model.predict_proba(newX)[:,1]
print(sample_validation_data[['pred','pred_proba','safe_loans']])

    pred  pred_proba  safe_loans
19     1    0.658435           1
79    -1    0.463694           1
24    -1    0.352490          -1
41     1    0.792105          -1


**Quiz Question:** Which loan has the highest probability of being classified as a **safe loan**?

**Checkpoint:** Can you verify that for all the predictions with `probability >= 0.5`, the model predicted the label **+1**?

### Tricky predictions!

Now, we will explore something pretty interesting. For each row in the **sample_validation_data**, what is the probability (according to **small_model**) of a loan being classified as **safe**?

**Hint:** Set `output_type='probability'` to make **probability** predictions using **small_model** on `sample_validation_data`:

In [33]:
small_model.predict_proba(newX)[:,1]

array([ 0.58103415,  0.40744661,  0.40744661,  0.76879888])

**Quiz Question:** Notice that the probability preditions are the **exact same** for the 2nd and 3rd loans. Why would this happen?

## Visualize the prediction on a tree


Note that you should be able to look at the small tree, traverse it yourself, and visualize the prediction being made. Consider the following point in the **sample_validation_data**

In [36]:
sample_validation_data.iloc[1,:]

short_emp                      0.000000
emp_length_num                10.000000
dti                           16.850000
last_delinq_none               1.000000
last_major_derog_none          1.000000
revol_util                    96.400000
total_rec_late_fee             0.000000
safe_loans                     1.000000
grade_A                        0.000000
grade_B                        0.000000
grade_C                        0.000000
grade_D                        1.000000
grade_E                        0.000000
grade_F                        0.000000
grade_G                        0.000000
sub_grade_A1                   0.000000
sub_grade_A2                   0.000000
sub_grade_A3                   0.000000
sub_grade_A4                   0.000000
sub_grade_A5                   0.000000
sub_grade_B1                   0.000000
sub_grade_B2                   0.000000
sub_grade_B3                   0.000000
sub_grade_B4                   0.000000
sub_grade_B5                   0.000000


Let's visualize the small tree here to do the traversing for this data point.

In [42]:
small_model.predict(newX)[1]

-1

**Note:** In the tree visualization above, the values at the leaf nodes are not class predictions but scores (a slightly advanced concept that is out of the scope of this course). You can read more about this [here](https://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf).  If the score is $\geq$ 0, the class +1 is predicted.  Otherwise, if the score < 0, we predict class -1.


**Quiz Question:** Based on the visualized tree, what prediction would you make for this data point?

Now, let's verify your prediction by examining the prediction made using GraphLab Create.  Use the `.predict` function on `small_model`.

# Evaluating accuracy of the decision tree model

Recall that the accuracy is defined as follows:
$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Let us start by evaluating the accuracy of the `small_model` and `decision_tree_model` on the training data

In [47]:
print(small_model.score(X,y))
print(decision_tree_model.score(X,y))


0.613502041694
0.640527616591


**Checkpoint:** You should see that the **small_model** performs worse than the **decision_tree_model** on the training data.


Now, let us evaluate the accuracy of the **small_model** and **decision_tree_model** on the entire **validation_data**, not just the subsample considered above.

In [49]:
newX=validation_data.ix[:,validation_data.columns!=target].as_matrix()
newy=validation_data[target].as_matrix()
print(small_model.score(newX,newy))
print(decision_tree_model.score(newX,newy))

0.619345109866
0.636148211978


**Quiz Question:** What is the accuracy of `decision_tree_model` on the validation set, rounded to the nearest .01?

## Evaluating accuracy of a complex decision tree model

Here, we will train a large decision tree with `max_depth=10`. This will allow the learned tree to become very deep, and result in a very complex model. Recall that in lecture, we prefer simpler models with similar predictive power. This will be an example of a more complicated model which has similar predictive power, i.e. something we don't want.

In [50]:
big_model=DecisionTreeClassifier(max_depth=10)
big_model.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Now, let us evaluate **big_model** on the training set and validation set.

In [51]:
print(big_model.score(X,y))
print(big_model.score(newX,newy))

0.66379217709
0.626669538992


**Checkpoint:** We should see that **big_model** has even better performance on the training set than **decision_tree_model** did on the training set.

**Quiz Question:** How does the performance of **big_model** on the validation set compare to **decision_tree_model** on the validation set? Is this a sign of overfitting?

### Quantifying the cost of mistakes

Every mistake the model makes costs money. In this section, we will try and quantify the cost of each mistake made by the model.

Assume the following:

* **False negatives**: Loans that were actually safe but were predicted to be risky. This results in an oppurtunity cost of losing a loan that would have otherwise been accepted. 
* **False positives**: Loans that were actually risky but were predicted to be safe. These are much more expensive because it results in a risky loan being given. 
* **Correct predictions**: All correct predictions don't typically incur any cost.


Let's write code that can compute the cost of mistakes made by the model. Complete the following 4 steps:
1. First, let us compute the predictions made by the model.
1. Second, compute the number of false positives.
2. Third, compute the number of false negatives.
3. Finally, compute the cost of mistakes made by the model by adding up the costs of true positives and false positives.

First, let us make predictions on `validation_data` using the `decision_tree_model`:

In [53]:
predictions = decision_tree_model.predict(newX)

array([-1,  1, -1, ..., -1, -1,  1])

**False positives** are predictions where the model predicts +1 but the true label is -1. Complete the following code block for the number of false positives:

In [68]:
validation_data['pred']=predictions
validation_data[['pred','safe_loans']].head()
FP=validation_data[validation_data['pred']==1][validation_data['safe_loans']==-1]
numFP=len(FP)

/usr/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/lib/python3.4/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


**False negatives** are predictions where the model predicts -1 but the true label is +1. Complete the following code block for the number of false negatives:

In [69]:
FN=validation_data[validation_data['pred']==-1][validation_data['safe_loans']==1]
numFN=len(FN)

/usr/lib/python3.4/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


**Quiz Question:** Let us assume that each mistake costs money:
* Assume a cost of \$10,000 per false negative.
* Assume a cost of \$20,000 per false positive.

What is the total cost of mistakes made by `decision_tree_model` on `validation_data`?

In [70]:
cost=10000*numFN+20000*numFP
print(cost)

50390000
